# **<mark>Download<mark>**
## "Download"腳本能擷取StorageServer內的dicom檔數據，並根據放入的資料庫DataBase下載其中的dicom
StorageServer內的dicom檔案分為由mini-PACS接收來自scanners的"dcm4chee server"內的原始資料Raw data(含個資)以及由豐康協助建立的"dicom server"的去名化影像anonymous data(無個資僅含病歷號)，兩者都可以使用"Download"腳本來下載資料庫內的影像，無論從哪個來源下載的都是去連結的dicom，並且會另外給予您下載的影像清單sorting_list，其中包含RIS號碼與病歷號的對照

## 使用說明:  
1.請將所有<font color=blue>下載用的資料表filted_DataBase_xxx</font>放到<font color=#0099ff>input資料夾</font>內，
腳本會將<font color=#0099ff>input資料夾</font>內的<font color=blue>所有資料表</font>合併並對其中的資料進行去連結  
2.下載的影像會儲存至您指定的<font color=green>存檔位置</font>，並產生<font color=green>下載的影像清單sorting_list</font>  
3.下載前請閱讀注意事項  
4.Raw data中有些是非影像檔(例如SR, PR)，而Download腳本僅能下載dicom影像，因此獲取的影像檔數量可能小於或等於<font color=green>影像清單sorting_list</font>內的Size

## <font color=red>注意事項:
<font color=red>★ 執行下載前請務必先以"Filter"腳本進行篩選  
★ 請確保您指定的存檔位置有足夠的空間  
★ 腳本請依序使用以減少錯誤: 以"DataBase"腳本建立對應的影像資料庫>>以"Filter"腳本進行篩選(以免資料量過於龐大)>>以"Download"腳本能進行下載  
★ 腳本執行後會紀錄上次的執行內容，若發生錯誤，請找到原始的檔案DataDownload.zip，解壓縮後置換換掉整個DataDownload資料夾(詳細方法請參考操作說明-Servers)  
★ 置換DataDownload資料夾前請備份您的重要資料表  
★ 腳本產生的檔案中，<font color=violet>資料庫檔案DataBase_xxx</font>是給腳本使用的資料表，而<font color=violet>影像清單sorting_list</font>則是方便給使用者查看影像內容的資料表，
    使用腳本時請務必放入正確的檔案  </font>

# **<font color=red>腳本使用方法:<p><font color=red>滑鼠選擇要執行的對話框，使對話框左邊顯示<font color=blue>藍色粗邊框<font color=red>後，按下shift+enter執行該對話框</font>**

# Step 1: 匯入功能包
(匯入成功會有提示)

In [1]:
#Step 1: import packages  (選取本對話框並按下ctrl+enter執行)
from utils.Functions import *
from utils.get import get_download_database,get_download_dicom,get_save_file_name  #2022/7/13 update
from utils import *

DEFAULT PARAMETERS IMPROT SCCESS!
對話框回報內容: 紅色標示注意事項；亮紫色提示類別；藍色標示input資料夾及input detail資料內容；綠色標示output資料；粉紅色背景表示程式碼錯誤
20220127 17:34:29 PACKAGES IMPORT SUCCESS!
請執行下一個步驟


# Step 2: 請先將篩選好要下載的影像資料表DataBase放到input資料夾中，
# 接著在以下對話框中設定條件
(可直接將資料表csv檔拖曳至左側的input資料夾)  
(雙擊對話框可進行編輯)  
(執行Step 2對話框後，請確認參數內容，無誤則執行Step 3，參數錯誤請修改並重新執行Step 2)

In [5]:
#### Step 2: 請設定去連結影像的儲存位置，設定完請按下ctrl+enter執行本對話框
download_folder=Storage_C  #請選擇存在USB裝置或StorageServer的C槽做為暫存去連結影像的位置  options={USB,Storage_C}

#底下這行不用改
print(check_DB.format(BlueLight.format(input_path),'',PurpleLight.format(unlink)),Red.format(check_above),sep='\n')

資料路徑: input
即將建立  去連結 的資料庫
請確認上述內容無誤後再執行下一步驟


# Step 3: 根據放入的資料表DataBase，從影像資料庫中下載檔案，並產生下載清單download_sorting_list
(執行Step 3對話框後會出現進度條，請務必等待進度完成，結束後會有 <font color=red>執行結束!</font> 的提示)

In [6]:
#Step 3: unlink dicom and save to assinged download folder, at the same time create and download sorting_list  (選取本對話框並按下ctrl+enter執行)
#optional settings:
# input_path='input'  #[optional]輸入路徑，篩選前資料庫DataBase的資料表的存放位置
# dicom_point='../storage'  #[optional]資料路徑，StorageServer絕對路徑/dicom的資料夾掛載的節點
# download_folder=Storage_C  #儲存路徑，可自行輸入儲存影像的位置 (需注意資料夾權限)

#define output file names:  #2022/7/13 update
save_download_csv_name,save_download_sorting_csv_name=get_save_file_name(output_path,Prefix='download_')
[download_path]=get_save_file_name(download_folder,Save_file=['Dicom'],Filetype='')  #下載路徑(Dicom-timestamp)，去連結的dicom的存檔位置
download_path=download_path[:-1]
print('saving:',save_download_csv_name,save_download_sorting_csv_name,download_path,waiting)

#讀取下載資料表並置換I/O路徑，產生download_DataBase:
S=time.time()
DB_path_list=GetInputDir(input_path,filetype='.csv')
for d in DB_path_list:   #jupyter會自動建立checkpoints導致混亂QQ
    if 'ipynb_checkpoints' in d:
        DB_path_list.remove(d)
print(Blue.format('input資料表={}'.format(DB_path_list)))
DB=GetConcatedDF(DB_path_list,cols=False)  #合併的資料表
print(total.format(len(DB),'download dicom'),'正在'+DB_build,waiting)
download_DB=get_download_database(Input_DB=DB,Dicom_point=dicom_point,Download_path=download_path,Save_csv_name=False)  #置換下載資料表的I/O
E=time.time()
t=E-S
print(time_spend.format('已完成{}'.format(DB_build),cut_float.format(2) % (t/3600),'hr'))

#儲存去連結影像:
S=time.time()
get_download_dicom(Input_DB=download_DB)
E=time.time()
print(time_spend.format('已完成{}'.format(unlink+'dicom'),cut_float.format(2) % (t/3600),'hr'))

#建立下載影像清單download_sorting_list:
Input_DB=download_DB
Output_sorting_list=GetUnique(Input_DB,df_sorting)
Output_sorting_list.to_csv(save_download_sorting_csv_name,index=0)
print(save.format('下載清單download_'+sort_build+': {} {}'.format(Output_sorting_list.shape,GreenItalic.format(save_download_sorting_csv_name))),Red.format(done))

100it [00:00, 1631.75it/s]
0it [00:00, ?it/s]

input資料表=['input/t.csv']
共有 100 個download dicom檔案 正在建立資料庫DataBase 請稍等...
已完成建立資料庫DataBase，建立費時:0.00 hr


100it [00:04, 22.33it/s]

已完成去連結dicom，建立費時:0.00 hr
下載清單download_sorting_list已建立: (3, 12) output/download_sorting_list-20220127173723.csv saved 
執行結束!


# Step 4: 請至您指定的<font color=green>存檔位置</font>查看檔案

## \~底下沒東西了~

<font size="4">版本資訊: <font size="3">ver.220715</font><br><font size="2">
    2022/7/15:  
    1.調整說明文字及順序  
    <br>
</font> 